<a href="https://colab.research.google.com/github/pandemic-tracking/global-vaccine/blob/main/OWID_WHO_vaccine_administration_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OWID/WHO comparison

In [2]:
import pandas as pd
from datetime import datetime
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [3]:
def print_bold(input):
	print('\033[1m{:10s}\033[0m'.format(input))

# Prepare dataset

In [4]:
# get OWID vaccination timeseries from Github
owid_data = pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv")

# drop non cumulative columns from OWID data
owid_data.drop(columns=[
    'daily_vaccinations','total_vaccinations_per_hundred','people_vaccinated_per_hundred','people_fully_vaccinated_per_hundred','total_boosters_per_hundred','daily_vaccinations_per_million',
    'daily_people_vaccinated','daily_people_vaccinated_per_hundred','daily_vaccinations_raw'], inplace = True)

# forward fill empty values in owid dataset
owid_data[['total_vaccinations','people_vaccinated','total_boosters']] = owid_data.groupby('iso_code')[['total_vaccinations','people_vaccinated','total_boosters']].apply(lambda x: x.fillna(method='ffill'))


# get latest date for each country in OWID dataset
latest_owid_dates = owid_data.groupby('iso_code')['date'].max().to_frame()

# merge full date 
latest_owid_data = pd.merge(latest_owid_dates,owid_data,on=['iso_code','date'])

# get WHO data
who_data = pd.read_csv("https://covid19.who.int/who-data/vaccination-data.csv")

# get PTC owid source classifications
owid_sources = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vTDKyIaQVtTIy7kn5pD2W8oKM3YoX3YOdSsH3q-r0INH2axjQl6YxgDHBi4HikKx_cmRElde_E-2vlr/pub?gid=2040574494&single=true&output=csv").filter(['Code','OWID Vax Source Category'])

# merge latest OWID data with WHO data
merged_data = pd.merge(latest_owid_data,who_data, how='inner', left_on='iso_code', right_on='ISO3').drop(columns=[
    'WHO_REGION','TOTAL_VACCINATIONS_PER100', 'PERSONS_VACCINATED_1PLUS_DOSE_PER100','PERSONS_FULLY_VACCINATED_PER100',
    'VACCINES_USED', 'FIRST_VACCINE_DATE', 'NUMBER_VACCINES_TYPES_USED', 'PERSONS_BOOSTER_ADD_DOSE_PER100',
    'PERSONS_BOOSTER_ADD_DOSE','PERSONS_VACCINATED_1PLUS_DOSE','PERSONS_FULLY_VACCINATED','people_vaccinated'
    ])
merged_data.rename(columns = {'date':'owid_date','DATE_UPDATED':'WHO_DATE',
'total_vaccinations':'owid_total_vaccinations',
'people_vaccinated':'owid_people_vaccinated',
'TOTAL_VACCINATIONS':'WHO_TOTAL_VACCINATIONS',
'PERSONS_VACCINATED_1PLUS_DOSE':'WHO_PERSONS_VACCINATED_1PLUS_DOSE',
'PERSONS_FULLY_VACCINATED':'WHO_PERSONS_FULLY_VACCINATED',
}, inplace = True)

# merge combined OWID+WHO data source with PTC owid source classifications
merged_data = pd.merge(merged_data,owid_sources, how='inner', left_on='ISO3', right_on='Code')

# calculate total vaccines diff
merged_data['diff_total_vaccinations'] = merged_data.WHO_TOTAL_VACCINATIONS - merged_data.owid_total_vaccinations
print_bold('Diff = WHO - owid' )
merged_data





Diff = WHO - owid


,iso_code,owid_date,location,owid_total_vaccinations,people_fully_vaccinated,total_boosters,COUNTRY,ISO3,DATA_SOURCE,WHO_DATE,WHO_TOTAL_VACCINATIONS,Code,OWID Vax Source Category,diff_total_vaccinations
0,ABW,2022-10-26,Aruba,173713.0,83757.0,NaN,Aruba,ABW,REPORTING,2022-10-21,173634.0,ABW,Country,-79.0
1,AFG,2022-10-24,Afghanistan,12055358.0,10386823.0,NaN,Afghanistan,AFG,REPORTING,2022-10-24,12055358.0,AFG,World Health Organization,0.0
2,AGO,2022-10-16,Angola,22814412.0,7814121.0,1127156.0,Angola,AGO,REPORTING,2022-10-16,22814412.0,AGO,World Health Organization,0.0
3,AIA,2022-10-21,Anguilla,24434.0,10366.0,2998.0,Anguilla,AIA,REPORTING,2022-10-21,24434.0,AIA,Pan American Health Organization,0.0
4,ALB,2022-10-16,Albania,2991576.0,1265900.0,363122.0,Albania,ALB,REPORTING,2022-10-16,2991576.0,ALB,World Health Organization,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,WSM,2022-10-17,Samoa,448674.0,177605.0,78912.0,Samoa,WSM,REPORTING,2022-09-26,440413.0,WSM,Pacific Data Hub (PDH),-8261.0
210,YEM,2022-10-20,Yemen,1139242.0,669393.0,54985.0,Yemen,YEM,REPORTING,2022-10-20,1139242.0,YEM,World Health Organization,0.0
211,ZAF,2022-10-26,South Africa,37775735.0,19457819.0,3783050.0,South Africa,ZAF,REPORTING,2022-10-16,37716831.0,ZAF,Country,-58904.0
212,ZMB,2022-10-25,Zambia,11907343.0,8005103.0,33684.0,Zambia,ZMB,REPORTING,2022-10-16,9315275.0,ZMB,Country,-2592068.0


# Using the merged dataset 

In [5]:
print_bold('\nTotal countries')
print(len( merged_data))

print_bold('\nDates')
dates_matching = len( merged_data.loc[merged_data['owid_date'] == merged_data['WHO_DATE']])
dates_owid_greater = len(merged_data.loc[merged_data['owid_date'] > merged_data['WHO_DATE']])
dates_owid_lesser = len(merged_data.loc[merged_data['owid_date'] < merged_data['WHO_DATE']])
print('Matching dates:' , dates_matching, 'countries')
print('owid greater ' , dates_owid_greater)
print('owid lesser ' , dates_owid_lesser)


matching_total_vaccinations_df = merged_data.loc[merged_data['owid_total_vaccinations'] == merged_data['WHO_TOTAL_VACCINATIONS']]
totalvax_matching = len( matching_total_vaccinations_df.index)
total_matching_vax_who = int(matching_total_vaccinations_df['WHO_TOTAL_VACCINATIONS'].sum(axis=0))
total_matching_vax_owid = int(matching_total_vaccinations_df['owid_total_vaccinations'].sum(axis=0))
print_bold('\nMatching total vaccinations: ' + str(totalvax_matching) + ' countries')
print('WHO total doses: %s' % "{:,}".format(total_matching_vax_who))
print('OWID total doses: %s' % "{:,}".format(total_matching_vax_owid))

# owid greater

owid_greater_total_vaccinations_df = merged_data.loc[merged_data['owid_total_vaccinations'] > merged_data['WHO_TOTAL_VACCINATIONS']]
owid_greater_count = len(owid_greater_total_vaccinations_df)
owid_greater_who_totalvax = int(owid_greater_total_vaccinations_df['WHO_TOTAL_VACCINATIONS'].sum(axis=0))
owid_greater_owid_totalvax = int(owid_greater_total_vaccinations_df['owid_total_vaccinations'].sum(axis=0))

print_bold('\nOWID greater total doses: ' + str(owid_greater_count) +  ' countries')
print('WHO total doses: %s' % "{:,}".format(owid_greater_who_totalvax))
print('OWID total doses: %s' % "{:,}".format(owid_greater_owid_totalvax))
print('DIFF: %s' % "{:,}".format(owid_greater_owid_totalvax-owid_greater_who_totalvax))

# owid lesser
owid_lesser_total_vaccinations_df = merged_data.loc[merged_data['owid_total_vaccinations'] < merged_data['WHO_TOTAL_VACCINATIONS']]
owid_lesser_count = len(owid_lesser_total_vaccinations_df)
owid_lesser_who_totalvax = int(owid_lesser_total_vaccinations_df['WHO_TOTAL_VACCINATIONS'].sum(axis=0))
owid_lesser_owid_totalvax = int(owid_lesser_total_vaccinations_df['owid_total_vaccinations'].sum(axis=0))

print_bold('\nOWID lesser total doses: '  +  str(owid_lesser_count) +  ' countries')
print('WHO total doses: %s' % "{:,}".format(owid_lesser_who_totalvax))
print('OWID total doses: %s' % "{:,}".format(owid_lesser_owid_totalvax))
print('DIFF: %s' % "{:,}".format(owid_lesser_who_totalvax-owid_lesser_owid_totalvax))


print_bold('\nTotal Doses')
totalvax_who = int(merged_data['WHO_TOTAL_VACCINATIONS'].sum(axis=0))
total_vax_owid= int(merged_data['owid_total_vaccinations'].sum(axis=0))
total_vax_diff = totalvax_who-total_vax_owid
print('WHO total doses: %s' % "{:,}".format(totalvax_who))
print('OWID total doses: %s' % "{:,}".format(total_vax_owid))
print('Overall diff total doses (WHO-Owid): %s' % "{:,}".format(total_vax_diff))

# dictionary of lists
dict = {
        'Date' : [pd.Timestamp.now(tz = 'US/Eastern')],
        'Total countries':[len( merged_data)],
        'Dates - matching': [dates_matching],
        'Dates - OWID greater':[dates_owid_greater],
        'Dates - OWID lesser':[dates_owid_lesser],
        'Total Vax - matching': [totalvax_matching],
        'Total Vax - matching- WHO total':[total_matching_vax_who],
        'Total Vax - matching- OWID total':[total_matching_vax_owid],
        'Total Vax - OWID greater - count':[owid_greater_count],
        'Total Vax - OWID greater - WHO total':[owid_greater_who_totalvax],
        'Total Vax - OWID greater - OWID total':[owid_greater_owid_totalvax],
        'Total Vax - OWID greater - Diff':[owid_greater_owid_totalvax-owid_greater_who_totalvax],
        'Total Vax - OWID lesser - count':[owid_lesser_count],
        'Total Vax - OWID lesser - WHO total':[owid_lesser_who_totalvax],
        'Total Vax - OWID lesser - OWID total':[owid_lesser_owid_totalvax],
        'Total Vax - OWID lesser - Diff':[owid_lesser_who_totalvax-owid_lesser_owid_totalvax],
        'Total Vax - WHO total':[totalvax_who],
        'Total Vax - OWID total':[total_vax_owid],
        'Total Vax - Diff':[total_vax_diff],

        }

 
df = pd.DataFrame(dict)
 
df



Total countries
214

Dates    
Matching dates: 107 countries
owid greater  63
owid lesser  44

Matching total vaccinations: 121 countries
WHO total doses: 2,851,022,554
OWID total doses: 2,851,022,554

OWID greater total doses: 59 countries
WHO total doses: 2,465,845,703
OWID total doses: 2,514,807,053
DIFF: 48,961,350

OWID lesser total doses: 34 countries
WHO total doses: 7,501,921,475
OWID total doses: 7,420,668,790
DIFF: 81,252,685

Total Doses
WHO total doses: 12,818,789,732
OWID total doses: 12,786,498,397
Overall diff total doses (WHO-Owid): 32,291,335


,Date,Total countries,Dates - matching,Dates - OWID greater,Dates - OWID lesser,Total Vax - matching,Total Vax - matching- WHO total,Total Vax - matching- OWID total,Total Vax - OWID greater - count,Total Vax - OWID greater - WHO total,Total Vax - OWID greater - OWID total,Total Vax - OWID greater - Diff,Total Vax - OWID lesser - count,Total Vax - OWID lesser - WHO total,Total Vax - OWID lesser - OWID total,Total Vax - OWID lesser - Diff,Total Vax - WHO total,Total Vax - OWID total,Total Vax - Diff
0,2022-10-27 16:05:05.143636-04:00,214,107,63,44,121,2851022554,2851022554,59,2465845703,2514807053,48961350,34,7501921475,7420668790,81252685,12818789732,12786498397,32291335


# Isolate anomalies

In [6]:
# OWID source is WHO but OWID total vaccinations do not match WHO
og = merged_data.loc[
          (merged_data["OWID Vax Source Category"] == 'World Health Organization') &
          (merged_data.diff_total_vaccinations!=0) 
].drop(columns=['people_fully_vaccinated','total_boosters','COUNTRY','ISO3','DATA_SOURCE','Code','OWID Vax Source Category']).sort_values(by=['diff_total_vaccinations'])
print('found mismatches: ', len(og))
og

found mismatches:  3


,iso_code,owid_date,location,owid_total_vaccinations,WHO_DATE,WHO_TOTAL_VACCINATIONS,diff_total_vaccinations
130,MOZ,2022-10-16,Mozambique,24676660.0,2022-10-16,19387285.0,-5289375.0
194,TUN,2022-10-18,Tunisia,14827155.0,2022-10-18,13150936.0,-1676219.0
114,LSO,2022-07-17,Lesotho,1077116.0,2022-08-14,1102069.0,24953.0


# Inferring WHO sources

In [7]:
def inferred_source(owid_source, who_source, diff_total_vaccinations):
  if ((owid_source == 'Country') or ((owid_source == 'Other'))) and (diff_total_vaccinations==0):
    return "Maybe Public Reporting"
  elif (owid_source == 'Country') and (who_source == 'REPORTING')  and (diff_total_vaccinations!=0):
    return "Direct Reporting to WHO"
  else:
    return 'Unknown'

merged_data['who_inferred_source'] = merged_data.apply(lambda x: inferred_source(x['OWID Vax Source Category'], x['DATA_SOURCE'], x['diff_total_vaccinations']), axis=1)
merged_data

,iso_code,owid_date,location,owid_total_vaccinations,people_fully_vaccinated,total_boosters,COUNTRY,ISO3,DATA_SOURCE,WHO_DATE,WHO_TOTAL_VACCINATIONS,Code,OWID Vax Source Category,diff_total_vaccinations,who_inferred_source
0,ABW,2022-10-26,Aruba,173713.0,83757.0,NaN,Aruba,ABW,REPORTING,2022-10-21,173634.0,ABW,Country,-79.0,Direct Reporting to WHO
1,AFG,2022-10-24,Afghanistan,12055358.0,10386823.0,NaN,Afghanistan,AFG,REPORTING,2022-10-24,12055358.0,AFG,World Health Organization,0.0,Unknown
2,AGO,2022-10-16,Angola,22814412.0,7814121.0,1127156.0,Angola,AGO,REPORTING,2022-10-16,22814412.0,AGO,World Health Organization,0.0,Unknown
3,AIA,2022-10-21,Anguilla,24434.0,10366.0,2998.0,Anguilla,AIA,REPORTING,2022-10-21,24434.0,AIA,Pan American Health Organization,0.0,Unknown
4,ALB,2022-10-16,Albania,2991576.0,1265900.0,363122.0,Albania,ALB,REPORTING,2022-10-16,2991576.0,ALB,World Health Organization,0.0,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,WSM,2022-10-17,Samoa,448674.0,177605.0,78912.0,Samoa,WSM,REPORTING,2022-09-26,440413.0,WSM,Pacific Data Hub (PDH),-8261.0,Unknown
210,YEM,2022-10-20,Yemen,1139242.0,669393.0,54985.0,Yemen,YEM,REPORTING,2022-10-20,1139242.0,YEM,World Health Organization,0.0,Unknown
211,ZAF,2022-10-26,South Africa,37775735.0,19457819.0,3783050.0,South Africa,ZAF,REPORTING,2022-10-16,37716831.0,ZAF,Country,-58904.0,Direct Reporting to WHO
212,ZMB,2022-10-25,Zambia,11907343.0,8005103.0,33684.0,Zambia,ZMB,REPORTING,2022-10-16,9315275.0,ZMB,Country,-2592068.0,Direct Reporting to WHO


In [8]:
import altair as alt
alt.Chart(merged_data).mark_bar().encode(
    y=alt.Y('who_inferred_source'),
    x='count()',
    tooltip='count()'
)

alt.Chart(...)

# SCRAP: UNDP DATA EXPLORATION
UNDP data is updated once a week, so during the week, they are lagging the WHO numbers

In [9]:
print('UNDP data download linked in https://github.com/UNDP-Data/Vaccine-Equity-Dashboard-Data')
undp_current_csv = "https://raw.githubusercontent.com/UNDP-Data/Vaccine-Equity-Dashboard-Data/main/Data.csv"
who_yesterday_csv = "http://pandemic-tracking-collective-data.s3-website-us-east-1.amazonaws.com/globalvax/GL/GL-who_vaccination_data-20221026-134504.csv"
undp_data = pd.read_csv(undp_current_csv)

# drop columns from UNDP data
undp_data.drop(columns=[
   'y2021GGXWDG_NGDP','vaccinesdelivered_lag',
       'y2021NGDPDPC', 'hdi', 'un_population2020',
       'informcovid19risk', 'covid19riskclass', 'gap_percapita', 'gap_gghe',
       'uhc', 'che', 'h7_vaccinationpolicy', 'e4_internationalsupport',
       'h5_investmentinvaccines', 'stringencyindex_average',
       'economicsupportindex', 'h7_name', 
       'number_vaccines_types_used', 'first_vaccine_date',
       'deathscumulativetotal', 'deathsnewlyreportedinlast7days',
       'vaccine_willingness_monthly', 'v3_vaccinefinancialsupportsummar',
       'v3_name', 'population', 'total_boosters', 'total_boosters_per_hundred',
       'dec40target', 'dec40target_name', 'securedvaccineofpopulation',
       'total_donations', 'cost_gdp_income40', 'pricebracket_che40',
       'utilrate_lag', 'donationsoftotal', 'casescumulativetotalper100000pop',
       'casesnewlyreportedinlast7daysper', 'deathscumulativetotalper100000po',
       'deathsnewlyreportedinlast7dayspe','vaccines_used'], inplace = True)

# merged undp dataset with owid+who datasaet
undp_merged = pd.merge(merged_data,undp_data, how='inner', left_on='iso_code', right_on='iso3').drop(columns=['COUNTRY','ISO3','total_boosters','vaccinesdelivered','deliveredpopulation','persons_vaccinated_1plus_dose','who_inferred_source',
                                                                                                              'iso_code','who_inferred_source','Code','countryname','iso3','people_fully_vaccinated','persons_fully_vaccinated',
                                                                                                              'persons_vaccinated_1plus_dose_pe'
                                                                                                              ])
undp_merged['diff_undp_who_totalvax'] = undp_merged.WHO_TOTAL_VACCINATIONS - undp_merged.total_vaccinations
#undp_merged['diff_undp_owid_totalvax'] = undp_merged.owid_total_vaccinations - undp_merged.total_vaccinations
undp_merged.loc[ undp_merged['diff_undp_who_totalvax'] != 0 ]
undp_merged

UNDP data download linked in https://github.com/UNDP-Data/Vaccine-Equity-Dashboard-Data


,owid_date,location,owid_total_vaccinations,DATA_SOURCE,WHO_DATE,WHO_TOTAL_VACCINATIONS,OWID Vax Source Category,diff_total_vaccinations,incomegroup,pop_total,total_vaccinations,total_vaccinations_per100,persons_fully_vaccinated_per100,diff_undp_who_totalvax
0,2022-10-26,Aruba,173713.0,REPORTING,2022-10-21,173634.0,Country,-79.0,High Income Economies,106766.0,173582.0,162.582000,78.389999,52.0
1,2022-10-24,Afghanistan,12055358.0,REPORTING,2022-10-24,12055358.0,World Health Organization,0.0,Low Income Economies,38928340.0,11977773.0,30.768999,26.531000,77585.0
2,2022-10-16,Angola,22814412.0,REPORTING,2022-10-16,22814412.0,World Health Organization,0.0,Lower Middle Income Economies,32866268.0,22696229.0,69.056000,23.775999,118183.0
3,2022-10-21,Anguilla,24434.0,REPORTING,2022-10-21,24434.0,Pan American Health Organization,0.0,Unclassified Economies,15002.0,24412.0,162.725010,68.950996,22.0
4,2022-10-16,Albania,2991576.0,REPORTING,2022-10-16,2991576.0,World Health Organization,0.0,Upper Middle Income Economies,2877800.0,2991576.0,104.000000,44.480999,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209,2022-10-17,Samoa,448674.0,REPORTING,2022-09-26,440413.0,Pacific Data Hub (PDH),-8261.0,Lower Middle Income Economies,198410.0,440413.0,221.967000,89.454002,0.0
210,2022-10-20,Yemen,1139242.0,REPORTING,2022-10-20,1139242.0,World Health Organization,0.0,Low Income Economies,29825968.0,1065395.0,3.572000,2.026000,73847.0
211,2022-10-26,South Africa,37775735.0,REPORTING,2022-10-16,37716831.0,Country,-58904.0,Upper Middle Income Economies,59308688.0,37679458.0,63.530998,35.123001,37373.0
212,2022-10-25,Zambia,11907343.0,REPORTING,2022-10-16,9315275.0,Country,-2592068.0,Lower Middle Income Economies,18383956.0,9315275.0,50.671001,32.872002,0.0


Today, 10/27, UNDP total vaccine numbers match the numbers from 10/25 

In [10]:
undp_current_csv = "https://raw.githubusercontent.com/UNDP-Data/Vaccine-Equity-Dashboard-Data/3cbebe982bc009595ab1b4f93c156fc9c815cf3f/Data.csv" # "https://raw.githubusercontent.com/UNDP-Data/Vaccine-Equity-Dashboard-Data/main/Data.csv"
who_older_csv = "http://pandemic-tracking-collective-data.s3-website-us-east-1.amazonaws.com/globalvax/merged/2022-10-26-17-50-04-234055-merged.csv"
who_older_csv = "http://pandemic-tracking-collective-data.s3-website-us-east-1.amazonaws.com/globalvax/merged/2022-10-25-17-50-04-270516-merged.csv"


undp_data = pd.read_csv(undp_current_csv)
who_older_data = pd.read_csv(who_older_csv)

# drop columns from UNDP data
undp_data.drop(columns=[
   'y2021GGXWDG_NGDP','vaccinesdelivered_lag',
       'y2021NGDPDPC', 'hdi', 'un_population2020',
       'informcovid19risk', 'covid19riskclass', 'gap_percapita', 'gap_gghe',
       'uhc', 'che', 'h7_vaccinationpolicy', 'e4_internationalsupport',
       'h5_investmentinvaccines', 'stringencyindex_average',
       'economicsupportindex', 'h7_name', 
       'number_vaccines_types_used', 'first_vaccine_date',
       'deathscumulativetotal', 'deathsnewlyreportedinlast7days',
       'vaccine_willingness_monthly', 'v3_vaccinefinancialsupportsummar',
       'v3_name', 'population', 'total_boosters', 'total_boosters_per_hundred',
       'dec40target', 'dec40target_name', 'securedvaccineofpopulation',
       'total_donations', 'cost_gdp_income40', 'pricebracket_che40',
       'utilrate_lag', 'donationsoftotal', 'casescumulativetotalper100000pop',
       'casesnewlyreportedinlast7daysper', 'deathscumulativetotalper100000po',
       'deathsnewlyreportedinlast7dayspe','vaccines_used'], inplace = True)

who_older_data.columns


# merged undp dataset with owid+who datasaet
undp_merged2 = pd.merge(who_older_data,undp_data, how='inner', left_on='ISO3', right_on='iso3').drop(columns=['total_boosters','people_fully_vaccinated','persons_fully_vaccinated','persons_vaccinated_1plus_dose','persons_vaccinated_1plus_dose_pe',
         'iso_code', 'countryname','countryname','iso3','deliveredpopulation','vaccinesdelivered','deliveredpopulation' ,'persons_fully_vaccinated_per100' ,'Unnamed: 0'  ,'COUNTRY'                                                                                             ''
                                                                                                             ''])
undp_merged2['diff_undp_who_totalvax'] = undp_merged2.WHO_TOTAL_VACCINATIONS - undp_merged2.total_vaccinations
undp_diff2 = undp_merged2.loc[ undp_merged2['diff_undp_who_totalvax'] != 0 ]
print('different: ', len(undp_diff2))
undp_diff2

different:  0


,owid_date,location,owid_total_vaccinations,ISO3,DATA_SOURCE,WHO_DATE,WHO_TOTAL_VACCINATIONS,Code,OWID Vax Source Category,diff_total_vaccinations,incomegroup,pop_total,total_vaccinations,total_vaccinations_per100,diff_undp_who_totalvax
